In [ ]:
import os

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import (
        SearchIndex,
        SearchField,
        SearchFieldDataType,
        SimpleField,
        SearchableField,
        VectorSearch,
        VectorSearchProfile,
        HnswAlgorithmConfiguration,
    )
import pandas as pd
from azure.core.exceptions import ResourceNotFoundError


In [ ]:
def get_all_search_indexes():
    # Create a SearchIndexClient
    credential = AzureKeyCredential(key)
    index_client = SearchIndexClient(service_endpoint, credential)
    
    # Get all indexes
    indexes = list(index_client.list_indexes())

    final_indexes = [index.name for index in indexes]
        
    return final_indexes

In [ ]:
def get_defect_documents():
    df = pd.read_excel("defect.xlsx")


    data_as_dict = df.to_dict(orient='records')
    df.columns = df.columns.str.replace(' ', '_')

    # Convert the DataFrame to a list of dictionaries
    data_as_dict = df.to_dict(orient='records')

    for item in data_as_dict:
        item["TitleVector"] = get_embeddings(item["Title"])

    return data_as_dict


In [ ]:
print("Retrieving all indexes...")
# get all indexes
all_indexes = get_all_search_indexes()
print(all_indexes)

In [ ]:
def get_bug_index(name: str):
    fields = [
        SimpleField(name="ID", type=SearchFieldDataType.String, key=True),
        SearchableField(name="Work_Item_Type", type=SearchFieldDataType.String, filterable=True, facetable=True),
        SearchableField(name="State", type=SearchFieldDataType.String, filterable=True, facetable=True),
        SearchableField(name="Area", type=SearchFieldDataType.String, filterable=True),
        SearchField(
            name="TitleVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=1536,
             vector_search_profile_name="my-vector-config",
    ),

        SearchableField(name="Title", type=SearchFieldDataType.String),
        SearchableField(name="Description", type=SearchFieldDataType.String),
        SearchableField(name="Repro_Steps", type=SearchFieldDataType.String),
        SearchableField(name="Created_Date", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="Created_By", type=SearchFieldDataType.String),
        SearchableField(name="Assigned_To", type=SearchFieldDataType.String),
        SearchableField(name="Iteration_Path", type=SearchFieldDataType.String),
        SearchableField(name="Severity", type=SearchFieldDataType.String, filterable=True, facetable=True),
        SearchableField(name="Priority", type=SearchFieldDataType.Double),
        SearchableField(name="Efforts", type=SearchFieldDataType.String),
        SearchableField(name="Comment", type=SearchFieldDataType.String),
        SearchableField(name="Closed_By", type=SearchFieldDataType.String),
        SearchableField(name="Closed_Date", type=SearchFieldDataType.String),
        SearchableField(name="Closing_Comment", type=SearchFieldDataType.String),
        SearchableField(name="Reason", type=SearchFieldDataType.String),
        SearchableField(name="Story_Points", type=SearchFieldDataType.String),
        SearchableField(name="Parent_Feature_Id", type=SearchFieldDataType.String),
        SearchableField(name="Parent_Feature_Title", type=SearchFieldDataType.String),
 ]

    vector_search = VectorSearch(
        profiles=[VectorSearchProfile(name="my-vector-config", algorithm_configuration_name="my-algorithms-config")],
        algorithms=[HnswAlgorithmConfiguration(name="my-algorithms-config")],
 )

    return SearchIndex(name=name, fields=fields, vector_search=vector_search)


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import openai
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Configure OpenAI


# Replace with your actual search service admin key
credential = AzureKeyCredential(key)


def get_embeddings(text: str):
    """Get embeddings for text using Azure OpenAI"""
    if not text or text.strip() == "":
        # Return a zero vector if text is empty
        return [0.0] * 1536  # text-embedding-ada-002 returns 1536 dimensions
    
    try:
        response = openai.Embedding.create(
            input=text.strip(), 
            engine="text-embedding-ada-002"
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error getting embeddings for text '{text[:50]}...': {str(e)}")
        # Return zero vector as fallback
        return [0.0] * 1536

def safe_value(val):
    """Safely convert values to strings, handling NaN and None"""
    if pd.isna(val) or val in [np.nan, None]:
        return ""
    if isinstance(val, (pd.Timestamp, datetime)):
        return val.isoformat()
    # Ensure we return a string and handle any potential JSON serialization issues
    try:
        result = str(val)
        # Remove any problematic characters that might cause JSON issues
        return result.replace('\x00', '').strip()
    except:
        return ""

def safe_numeric_value(val, default=0):
    """Safely convert values to numeric, handling NaN and None"""
    if pd.isna(val) or val in [np.nan, None]:
        return default
    try:
        return float(val) if val != "" else default
    except:
        return default

def get_defect_documents():
    """Process Excel file and return documents for Azure Search"""
    df = pd.read_excel("defect-new.xlsx")
    df.columns = df.columns.str.replace(' ', '_')
    data = df.to_dict(orient='records')

    transformed_data = []
    for i, item in enumerate(data):
        title = item.get('Title', 'No Title')
        print(f"Processing item {i+1}/{len(data)}: {title}")
        
        # Get embeddings for the title
        title_vector = get_embeddings(title)
        
        transformed_item = {
            "ID": safe_value(item.get("ID")),
            "Work_Item_Type": safe_value(item.get("Work_Item_Type")),
            "State": safe_value(item.get("State")),
            "Area": safe_value(item.get("Area")),
            "Title": safe_value(item.get("Title")),
            "Description": safe_value(item.get("Description")) or "No description provided",
            "Repro_Steps": safe_value(item.get("Repro_Steps")),
            "Created_Date": safe_value(item.get("Created_Date")),
            "Created_By": safe_value(item.get("Created_By")),
            "Assigned_To": safe_value(item.get("Assigned_To")),
            "Iteration_Path": safe_value(item.get("Iteration_Path")),
            "Severity": safe_value(item.get("Severity")),
            "Priority": safe_value(item.get("Priority")),
            "Efforts": safe_numeric_value(item.get("Efforts"), 0),
            "Comment": safe_value(item.get("Comment")),
            "Closed_By": safe_value(item.get("Closed_By")),
            "Closed_Date": safe_value(item.get("Closed_Date")),
            "Closing_Comment": safe_value(item.get("Closing_Comment")),
            "Reason": safe_value(item.get("Reason")),
            "Story_Points": safe_numeric_value(item.get("Story_Points"), 0),
            "Parent_Feature_Id": safe_value(item.get("Parent_Feature_Id")),
            "Parent_Feature_Title": safe_value(item.get("Parent_Feature_Title")),
            "TitleVector": title_vector
        }
        transformed_data.append(transformed_item)

    return transformed_data

def main():
    """Main function to upload documents to Azure Search"""
    try:
        # Create SearchClient
        print(f"Service endpoint: {service_endpoint}")
        print(f"Index name: {index_name}")
        client = SearchClient(service_endpoint, index_name, credential)
        
        # Get defect documents
        defect_docs = get_defect_documents()
        
        print("Uploading defect documents to the index...")
        
        # Upload documents in smaller batches to avoid size limits
        batch_size = 10  # Adjust based on your document size
        for i in range(0, len(defect_docs), batch_size):
            batch = defect_docs[i:i + batch_size]
            print(f"Uploading batch {i//batch_size + 1}/{(len(defect_docs) + batch_size - 1)//batch_size}")
            result = client.upload_documents(documents=batch)
            
            # Check for any failures
            for doc_result in result:
                if not doc_result.succeeded:
                    print(f"Failed to upload document {doc_result.key}: {doc_result.error_message}")
        
        print("Defect documents uploaded successfully.")
        
    except Exception as e:
        print(f"Error during upload: {str(e)}")
        print("Please check your credentials and index configuration.")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import openai
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential


credential = AzureKeyCredential(admin_key)

def get_embeddings(text: str):
    """Get embeddings for text using Azure OpenAI"""
    if not text or text.strip() == "":
        # Return a zero vector if text is empty
        return [0.0] * 1536  # text-embedding-ada-002 returns 1536 dimensions
    
    try:
        response = openai.Embedding.create(
            input=text.strip(), 
            engine="text-embedding-ada-002"
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error getting embeddings for text '{text[:50]}...': {str(e)}")
        # Return zero vector as fallback
        return [0.0] * 1536

def safe_value(val):
    """Safely convert values to strings, handling NaN and None"""
    if pd.isna(val) or val in [np.nan, None]:
        return ""
    if isinstance(val, (pd.Timestamp, datetime)):
        return val.isoformat()
    # Ensure we return a string and handle any potential JSON serialization issues
    try:
        result = str(val)
        # Remove any problematic characters that might cause JSON issues
        return result.replace('\x00', '').strip()
    except:
        return ""

def safe_numeric_value(val, default="0"):
    """Safely convert values to string representation of numeric, handling NaN and None"""
    if pd.isna(val) or val in [np.nan, None]:
        return default
    try:
        # Convert to float first to handle numeric values, then to string
        if val == "" or val == "":
            return default
        numeric_val = float(val)
        # Convert to int if it's a whole number, otherwise keep as float
        if numeric_val.is_integer():
            return str(int(numeric_val))
        else:
            return str(numeric_val)
    except:
        return default

def get_defect_documents():
    """Process Excel file and return documents for Azure Search"""
    df = pd.read_excel("defect.xlsx")
    df.columns = df.columns.str.replace(' ', '_')
    data = df.to_dict(orient='records')

    transformed_data = []
    for i, item in enumerate(data):
        title = item.get('Title', 'No Title')
        print(f"Processing item {i+1}/{len(data)}: {title}")
        
        # Get embeddings for the title
        title_vector = get_embeddings(title)
        
        transformed_item = {
            "ID": safe_value(item.get("ID")),
            "Work_Item_Type": safe_value(item.get("Work_Item_Type")),
            "State": safe_value(item.get("State")),
            "Area": safe_value(item.get("Area")),
            "Title": safe_value(item.get("Title")),
            "Description": safe_value(item.get("Description")) or "No description provided",
            "Repro_Steps": safe_value(item.get("Repro_Steps")),
            "Created_Date": safe_value(item.get("Created_Date")),
            "Created_By": safe_value(item.get("Created_By")),
            "Assigned_To": safe_value(item.get("Assigned_To")),
            "Iteration_Path": safe_value(item.get("Iteration_Path")),
            "Severity": safe_value(item.get("Severity")),
            "Priority": safe_value(item.get("Priority")),
            "Efforts": safe_numeric_value(item.get("Efforts"), "0"),
            "Comment": safe_value(item.get("Comment")),
            "Closed_By": safe_value(item.get("Closed_By")),
            "Closed_Date": safe_value(item.get("Closed_Date")),
            "Closing_Comment": safe_value(item.get("Closing_Comment")),
            "Reason": safe_value(item.get("Reason")),
            "Story_Points": safe_numeric_value(item.get("Story_Points"), "0"),
            "Parent_Feature_Id": safe_value(item.get("Parent_Feature_Id")),
            "Parent_Feature_Title": safe_value(item.get("Parent_Feature_Title")),
            "TitleVector": title_vector
        }
        transformed_data.append(transformed_item)

    return transformed_data

def main():
    """Main function to upload documents to Azure Search"""
    try:
        # Create SearchClient
        print(f"Service endpoint: {service_endpoint}")
        print(f"Index name: {index_name}")
        client = SearchClient(service_endpoint, index_name, credential)
        
        # Get defect documents
        defect_docs = get_defect_documents()
        
        print("Uploading defect documents to the index...")
        
        # Upload documents in smaller batches to avoid size limits
        batch_size = 10  # Adjust based on your document size
        for i in range(0, len(defect_docs), batch_size):
            batch = defect_docs[i:i + batch_size]
            print(f"Uploading batch {i//batch_size + 1}/{(len(defect_docs) + batch_size - 1)//batch_size}")
            result = client.upload_documents(documents=batch)
            
            # Check for any failures
            for doc_result in result:
                if not doc_result.succeeded:
                    print(f"Failed to upload document {doc_result.key}: {doc_result.error_message}")
        
        print("Defect documents uploaded successfully.")
        
    except Exception as e:
        print(f"Error during upload: {str(e)}")
        print("Please check your credentials and index configuration.")

if __name__ == "__main__":
    main()

In [ ]:
credential = AzureKeyCredential(key)
index_client = SearchIndexClient(service_endpoint, credential)
index = get_bug_index(index_name)
print(type(index))

print("Creating the index if it does not exist")
try:
    index_client.get_index(index.name)
    print(f"Index '{index.name}' already exists.")
except ResourceNotFoundError:
    index_client.create_index(index)
    print(f"Index '{index.name}' created successfully.")


print(f"Index {index.name} created successfully.")


In [ ]:
index_client.delete_index(index_name)

In [ ]:
index = get_bug_index(index_name)
try:
    index_client.get_index(index.name)
except ResourceNotFoundError:
    print("creating a new one")
    index_client.create_index(index)



In [ ]:
def get_existing_index_fields(index_name):
    """
    Get the existing index schema to understand current field structure
    """
    try:
        credential = AzureKeyCredential(key)
        index_client = SearchIndexClient(service_endpoint, credential)
        existing_index = index_client.get_index(index_name)
        
        print(f"Existing index '{index_name}' fields:")
        for field in index.fields:
            print(f"Name: {field.name}, Type: {field.type}, Key: {field.key}, Searchable: {field.searchable}, Filterable: {field.filterable}")

        
        return existing_index
    except Exception as e:
        print(f"Error getting existing index: {e}")
        return None

In [ ]:
get_existing_index_fields(index_name)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

def safe_value(val):
    if pd.isna(val) or val in [np.nan, None]:
        return ""
    if isinstance(val, (pd.Timestamp, datetime)):
        return val.isoformat()
    return str(val)

def get_defect_documents():
    df = pd.read_excel("defect-new.xlsx")
    df.columns = df.columns.str.replace(' ', '_')
    data = df.to_dict(orient='records')

    transformed_data = []
    for i, item in enumerate(data):
        print(f"Processing item {i+1}/{len(data)}: {item.get('Title', 'No Title')}")
        transformed_item = {
            "ID": safe_value(item.get("ID")),
            "Work_Item_Type": safe_value(item.get("Work_Item_Type")),
            "State": safe_value(item.get("State")),
            "Area": safe_value(item.get("Area")),
            "Title": safe_value(item.get("Title")),
            "Description": item.get("Description") or "No description provided",
            "Repro_Steps": safe_value(item.get("Repro_Steps")),
            "Created_Date": safe_value(item.get("Created_Date")),
            "Created_By": safe_value(item.get("Created_By")),
            "Assigned_To": safe_value(item.get("Assigned_To")),
            "Iteration_Path": safe_value(item.get("Iteration_Path")),
            "Severity": safe_value(item.get("Severity")),
            "Priority": safe_value(item.get("Priority")),
            "Efforts": item.get("Efforts") if pd.notna(item.get("Efforts")) else 0,
            "Comment": safe_value(item.get("Comment")),
            "Closed_By": safe_value(item.get("Closed_By")),
            "Closed_Date": safe_value(item.get("Closed_Date")),
            "Closing_Comment": safe_value(item.get("Closing_Comment")),
            "Reason": safe_value(item.get("Reason")),
            "Story_Points": item.get("Story_Points") if pd.notna(item.get("Story_Points")) else 0,
            "Parent_Feature_Id": safe_value(item.get("Parent_Feature_Id")),
            "Parent_Feature_Title": safe_value(item.get("Parent_Feature_Title")),
            "TitleVector": get_embeddings(item.get("Title", ""))
        }
        transformed_data.append(transformed_item)

    return transformed_data


In [ ]:
# Create a SearchClient to interact with the index
print(service_endpoint)
print(index_name)
client = SearchClient(service_endpoint, index_name, credential)
defect_docs = get_defect_documents()
print("Uploading defect documents to the index...")
# create embeddings
client.upload_documents(documents=defect_docs)

print("defects documents uploaded successfully.")

In [ ]:
def get_defect_documents():
    """
    Prepare defect data for indexing with proper field name mapping
    """
    data = [
        {
            "Work_Item_Type": "Bug",
            "ID": 897,
            "State": "Closed",
            "Area": "Gen AI QEP 2.0\\Create - Existing Solution Enhancements",
            "Title": "When a solution is created on a name which is already existing, the sysyem should give an api error alert. ",
            "Description": None,
            "Repro Steps": "<div><span style=\"display:inline !important;\">When a user creates a solution by a duplicate name, there should be an alert message &quot;Solution already exists&quot;.</span><br> </div>",
            "Created Date": "11/27/2023 5:43:52 PM",
            "Created By": None,
            "Assigned To": None,
            "Iteration Path": "Gen AI QEP 2.0\\PI-2.1",
            "Severity": "3 - Medium",
            "Priority": 2.0,
            "Efforts": None,
            "Comment": None,
            "Closed By": "Siva Reddy Dirisanapu <siva-reddy.dirisanapu@capgemini.com>",
            "Closed Date": "2023-01-12T16:25:11",
            "Closing Comment": None,
            "Reason": "Fixed and verified",
            "Story Points": None,
            "Parent Feature Id": 555.0,
            "Parent Feature Title": None
        }
    ]
    print(type(data))

    # Transform data to match existing index structure
    transformed_data = []
    for i, item in enumerate(data):
        transformed_item = {
            "ID": str(item["ID"]),
            "Work_Item_Type": item["Work_Item_Type"],
            "State": item["State"],
            "Area": item["Area"],
            "Title": item["Title"],
            "Description": item["Description"] or "No description provided",
            "Repro_Steps": item["Repro Steps"],
            "Created_Date": item["Created Date"],
            "Created_By": item["Created By"],
            "Assigned_To": item["Assigned To"],
            "Iteration_Path": item["Iteration Path"],
            "Severity": item["Severity"],
            "Priority": str(item["Priority"]),
            "Efforts": item["Efforts"],
            "Comment": item["Comment"],
            "Closed_By": item["Closed By"],
            "Closed_Date": item["Closed Date"],
            "Closing_Comment": item["Closing Comment"],
            "Reason": item["Reason"],
            "Story_Points": item["Story Points"],
            "Parent_Feature_Id": str(item["Parent Feature Id"]),
            "Parent_Feature_Title": item["Parent Feature Title"],
            "TitleVector": get_embeddings(item["Title"])  # Use existing vector field
        }
        transformed_data.append(transformed_item)

    return transformed_data

In [ ]:
# Create a SearchClient to interact with the index
print(service_endpoint)
print(index_name)
client = SearchClient(service_endpoint, index_name, credential)
defect_docs = get_defect_documents()
print(defect_docs)
print("Uploading defect documents to the index...")
# create embeddings
client.upload_documents(documents=defect_docs)

print("defects documents uploaded successfully.")

In [ ]:
def get_all_defect_documents():
    """
    Retrieves all defect documents from the Azure Cognitive Search index.
    """

    print(f"key is ........{key}")
    print(f"service_endpoint is ........{service_endpoint}")
    try:
        credential = AzureKeyCredential(key)
        search_client = SearchClient(service_endpoint, index_name, credential)

        # search_text="*" matches all documents
        results = search_client.search(
            search_text="*",
            select=["ID", "Title", "Description", "Severity", "State", "Work_Item_Type"],
            top=1000
        )

        documents = list(results)

        print(f"Retrieved {len(documents)} defect document(s).")
        for doc in documents:
            print(f"  - ID: {doc['ID']}, Title: {doc.get('Title', 'N/A')}, Severity: {doc.get('Severity', 'N/A')}, State: {doc.get('State', 'N/A')}")

        return documents

    except Exception as ex:
        print(f"Error retrieving defect documents: {ex}")
        raise

In [ ]:
def get_defect_documents_by_id(document_ids):
    """
    Retrieves defect documents by ID
    """
    try:
        credential = AzureKeyCredential(key)
        search_client = SearchClient(service_endpoint, index_name, credential)

        documents = []
        for doc_id in document_ids:
            try:
                doc = search_client.get_document(key=str(doc_id))
                documents.append(doc)
                print(f"  - ID: {doc['ID']}, Title: {doc.get('Title', 'N/A')}, Severity: {doc.get('Severity', 'N/A')}")
            except Exception as e:
                print(f"  - Could not find defect document with ID {doc_id}: {e}")

        return documents

    except Exception as ex:
        print(f"Error: {ex}")
        raise


In [ ]:
get_defect_documents_by_id(["897"])

In [ ]:
def single_vector_search_defects(query):
    """
    Perform vector search on defect titles
    """
    search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
    vector_query = VectorizedQuery(
        vector=get_embeddings(query), 
        k_nearest_neighbors=5, 
        fields="TitleVector"  # Use existing vector field
    )

    results = search_client.search(
        vector_queries=[vector_query],
        select=["ID", "Title", "Description", "Severity", "State"],
    )

    print(f"Vector search results for query: '{query}'")
    for result in results:
        print(f"  - ID: {result.get('ID', 'N/A')}, Title: {result.get('Title', 'N/A')}, Score: {result.get('@search.score', 'N/A')}")

    return list(results)

In [ ]:
single_vector_search_defects(query = "API error alert")

In [ ]:
def single_vector_search_with_filter_defects(query="API error", severity_filter="3 - Medium"):
    """
    Perform vector search with severity filter
    """
    search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
    vector_query = VectorizedQuery(
        vector=get_embeddings(query),
        k_nearest_neighbors=5,
        fields="TitleVector"  # Use existing vector field
    )

    results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        filter=f"Severity eq '{severity_filter}'",
        select=[ "ID", "Title", "Severity", "State"],
        top=5
    )

    print(f"Filtered vector search results for query: '{query}' with severity: '{severity_filter}'")
    for result in results:
        print(f"  - ID: {result.get('ID', 'N/A')}, Title: {result.get('Title', 'N/A')}, Severity: {result.get('Severity', 'N/A')}")

    return list(results)

In [ ]:
single_vector_search_with_filter_defects()

In [ ]:
def multi_query_hybrid_search_defects(queries):
    """
    Perform hybrid search for multiple defect-related queries
    """
    search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
    
    all_results = []
    
    for query in queries:
        vector_query = VectorizedQuery(
            vector=get_embeddings(query), 
            k_nearest_neighbors=3, 
            fields="TitleVector"  # Use existing vector field
        )
        
        results = search_client.search(
            search_text=query,
            vector_queries=[vector_query],
            select=["ID", "Title", "Severity", "State"],
            top=3
        )
        
        query_results = list(results)
        all_results.append({
            "query": query,
            "results": query_results
        })
        
        print(f"Results for defect query: '{query}'")
        for result in query_results:
            print(f"  - ID: {result.get('ID', 'N/A')}, Title: {result.get('Title', 'N/A')}, Severity: {result.get('Severity', 'N/A')}")
        print("---")
    
    return all_results

In [ ]:
multi_query_hybrid_search_defects([
    "What happens if a solution is created with a duplicate name?",
    "Is there an alert when trying to create an existing solution?",
    "Defect related to duplicate solution creation in Gen AI QEP 2.0 PI-2.1",
    "System behavior when solution name already exists",
    "Missing API error for duplicate solution creation",
    "Closed bugs with medium severity and fixed status in Gen AI QEP"
])

In [ ]:
def delete_defect_document(defect_id):
    """
    Delete a defect document by hotelId (the key field)
    """
    try:
        credential = AzureKeyCredential(key)
        search_client = SearchClient(service_endpoint, index_name, credential)
        
        search_client.delete_documents(documents=[{"hotelId": str(defect_id)}])
        print(f"Defect document with ID '{defect_id}' deleted.")
        
    except Exception as e:
        print(f"Error deleting defect document with ID '{defect_id}': {e}")

In [ ]:
def delete_index(index_name):
    """
    Delete the entire search index
    """
    credential = AzureKeyCredential(key)
    index_client = SearchIndexClient(service_endpoint, credential)

    try:
        index_client.delete_index(index_name)
        print(f"Index '{index_name}' deleted successfully.")
    except Exception as e:
        print(f"Error deleting index '{index_name}': {e}")